In [14]:
%load_ext autoreload
%autoreload 2

import awkward as ak
import numpy as np
import pandas as pd
from tabulate import tabulate

from python.analysis import shower_merging

from python.analysis import Master, Plots

events = Master.Data("work/ROOTFiles/v3/Prod4a_1GeV_BeamSim_00.root")
cuts = "csv/cuts/analysedCuts-1GeV.csv"
# cuts = "work/2023/Prod4a_1GeV_BeamSim_analysis/prod4a_merge_study/analysedCuts.csv"
shower_merging.EventSelection(events)
shower_merging.ValidPFOSelection(events)
start_showers, to_merge = shower_merging.SplitSample(events)

quantities = shower_merging.ShowerMergeQuantities(events, to_merge, cuts)
quantities.to_merge_dir = events.recoParticles.shower_direction
quantities.to_merge_pos = events.recoParticles.shower_start_pos


quantities.bestCut = "purity"

n_merge = -1

print("calculating score...")
def SortByStartingShower(data):
    return ak.concatenate([ak.unflatten(data[i], 1, -1) for i in range(2)], -1)

def ClosestQuantity(q : ak.Array, mask : ak.Array):
    masked_q = ak.where(mask, q, 9999999)
    q_to_merge = ak.argmin(masked_q, -1, keepdims=True)
    return ak.where(ak.min(masked_q, -1, keepdims=True) == 9999999, -1, q_to_merge)

#* retrieve quantities and find which start shower is closest to each PFO for each variable
quantities.Evaluate(events, start_showers)
print("evaluated quantities")
mask = SortByStartingShower(quantities.mask) # PFOs we want to merge after cut based selection is done

alpha = ClosestQuantity(SortByStartingShower(quantities.alpha), mask) # can use this to determine which starting shower the PFO is closest to in angle
x = ClosestQuantity(SortByStartingShower(quantities.delta_x), mask) # can use this to determine which starting shower the PFO is closest to in space
phi = ClosestQuantity(SortByStartingShower(quantities.delta_phi), mask) # can use this to determine which starting shower the PFO direction is most aligned to
print(mask)

#* figure out which is the common start shower between all variables
# if min phi, alpha and x are all the same then merge to that shower
# if two are the same, merge to the most common shower
# if none agree (shouldn't be possible)
#! should replace this with calculating the mode of the scores
scores = ak.sum(ak.concatenate([phi, x, alpha], -1), -1)
scores = ak.where(scores == 1, 0, scores) # [1, 0, 0]
scores = ak.where(scores == 2, 1, scores) # [1, 1, 0]
scores = ak.where(scores == 3, 1, scores) # [1, 1, 1]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
number of entries before|after HasTruthInfo: 23783|22882
number of entries before|after Pi0InFinalState: 23783|2433
number of entries before|after BeamMCFilter: 23783|2433


/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")


number of entries before|after PiBeamSelection: 2433|1722


number of dalitz decays: 32

number of entries before|after DiPhotonCut: 1722|1690
number of entries before|after RecoBeamParticleCut: 1690|1325
number of entries before|after HasPFO: 1325|1325
number of entries before|after HasBacktracked: 1325|1325
number of entries before|after BothPhotonsBacktracked: 1325|710


...

cutting on: d

cutting on: t

cutting on: p

'Evaluate' executed in 27.1287s

evaluated quantities
[[[False, False], [False, False], [False, ... [False, False], [False, False]]]


In [12]:
shower_merging.ShowerMergingPFOPerformance(events, start_showers, to_merge, scores, quantities)

mismatch (%): 1.1764705882352942

number of signal PFOs before cutting: 1625

number of background PFOs before cutting: 182759

╒═════════════════════════════════════════════╤════════════════╤═══════════════════════╤═══════════════════════════
═══════╤════════════════════════════════╕
│ PFO performance metric                      │ number of PFOs │ total efficiency (%)  │ signal/background 
efficiency (%) │ merged/unmerged efficiency (%) │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ signal PFOs merged and correctly matched    │ 168            │ 0.09111419645956265   │ 10.338461538461539        
│ 85.27918781725889              │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ signal PFOs merged and incorrectly matched  │ 2              │ 0.0010846928149947934 │ 0.12307692307692308       
│ 1.015228426395939              │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ background PFOs merged (false positive)     │ 27             │ 0.014643353002429712  │ 0.014773554243566665      
│ 13.705583756345177             │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ signal PFOs not merged (false negative)     │ 1455           │ 0.7891140229087122    │ 89.53846153846153         
│ 0.7899580317829162             │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ background PFOs not merged (true negatives) │ 182732         │ 99.1040437348143      │ 99.98522644575644         
│ 99.21004196821708              │
├─────────────────────────────────────────────┼────────────────┼───────────────────────┼───────────────────────────
───────┼────────────────────────────────┤
│ signal PFOs correctly matched               │ -              │ -                     │ 98.82352941176471         
│ 98.82352941176471              │
╘═════════════════════════════════════════════╧════════════════╧═══════════════════════╧═══════════════════════════
═══════╧════════════════════════════════╛

,0,1,2,3,4
0,PFO performance metric,number of PFOs,total efficiency (%),signal/background efficiency (%),merged/unmerged efficiency (%)
1,signal PFOs merged and correctly matched,168,0.091114,10.338462,85.279188
2,signal PFOs merged and incorrectly matched,2,0.001085,0.123077,1.015228
3,background PFOs merged (false positive),27,0.014643,0.014774,13.705584
4,signal PFOs not merged (false negative),1455,0.789114,89.538462,0.789958
5,background PFOs not merged (true negatives),182732,99.104044,99.985226,99.210042
6,signal PFOs correctly matched,-,-,98.823529,98.823529


In [13]:
shower_merging.ShowerMergingEventPerformance(events, start_showers, to_merge, scores)

number of events after selection: 710

number of events with PFOs to merge: 500

number of events where we merge: 134

number of events where we merge signal: 113

number of events where we merge background: 27

number of events where we merge signal and background 6

number of events where we merge only signal 107

number of events where we merge only background 21

╒══════════════════════════════╤══════════════════╤══════════════════╤════════════════════╕
│ performance metric           │ number of events │ total efficiency │ merging efficiency │
├──────────────────────────────┼──────────────────┼──────────────────┼────────────────────┤
│ signal merged                │ 113              │ 22.6             │ 84.32835820895522  │
├──────────────────────────────┼──────────────────┼──────────────────┼────────────────────┤
│ only signal merged           │ 107              │ 21.4             │ 79.85074626865672  │
├──────────────────────────────┼──────────────────┼──────────────────┼────────────────────┤
│ signal and background merged │ 6                │ 1.2              │ 4.477611940298507  │
╘══════════════════════════════╧══════════════════╧══════════════════╧════════════════════╛

,0,1,2,3
0,performance metric,number of events,total efficiency,merging efficiency
1,signal merged,113,22.6,84.328358
2,only signal merged,107,21.4,79.850746
3,signal and background merged,6,1.2,4.477612
